In [1]:
import os
import torch
import torchvision
import tarfile
import torch.nn as nn
import numpy as np
import torch.nn.functional as func
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torchvision.transforms as trans

In [2]:
 
download_url("https://s3.amazonaws.com/fast-ai-imageclas/cifar10.tgz", '.')

with tarfile.open('./cifar10.tgz', 'r:gz') as tar:
    tar.extractall(path='./data')
    
print(os.listdir('./data/cifar10'))
classes = os.listdir('./data/cifar10' + "/train")
print(classes)

  0%|          | 0/135107811 [00:00<?, ?it/s]

['train', 'test']
['frog', 'airplane', 'bird', 'truck', 'dog', 'deer', 'horse', 'automobile', 'cat', 'ship']


In [3]:
trainIF = ImageFolder('./data/cifar10/train', trans.Compose([trans.RandomCrop(32, padding=4, padding_mode='reflect'),
                         trans.AutoAugment(policy=trans.AutoAugmentPolicy.CIFAR10),
                         trans.RandomHorizontalFlip(),
                         trans.ToTensor(), 
                         trans.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010),inplace=True)]))
validIF = ImageFolder('./data/cifar10/test', trans.Compose([trans.ToTensor(), trans.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]))

In [4]:
def dohvati_uredaj():
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def na_uredaj(podatci, device):
    if isinstance(podatci, (list,tuple)):
        return [na_uredaj(i, device) for i in podatci]
    return podatci.to(device, non_blocking=True)

class DeviceDataLoader():
    def __init__(self, dl, uredaj):
        self.dl = dl
        self.uredaj = uredaj
        
    def __iter__(self):
        for i in self.dl: 
            yield na_uredaj(i, self.uredaj)

    def __len__(self):
        return len(self.dl)
    
uredaj = dohvati_uredaj()
uredaj

device(type='cuda')

In [5]:
batchSize = 400
trainDL = DataLoader(trainIF, batchSize, shuffle=True, num_workers = 2, pin_memory=True)
validDL = DataLoader(validIF, batchSize*2, num_workers = 2, pin_memory=True)

In [6]:
trainDL = DeviceDataLoader(trainDL, uredaj)
validDL = DeviceDataLoader(validDL, uredaj)

In [7]:
class SimpleResidualBlock(nn.Module):
    def __init__(self):
        super().__init__()
        channelsIn = 3
        channelsOut = 3
        self.conv1 = nn.Conv2d(channelsIn, channelsOut, 3, 1, 1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(channelsIn, channelsOut, 3, 1, 1)
        self.relu2 = nn.ReLU()
        
    def forward(self, i):
        izlaz = self.conv2(self.relu1(self.conv1(i)))
        return self.relu2(izlaz) + i 
    


simpleResnet = na_uredaj(SimpleResidualBlock(), uredaj)

for slike, oznake in trainDL:
    izlaz = simpleResnet(slike)
    print(izlaz.shape)
    break
    
del simpleResnet, slike, oznake
torch.cuda.empty_cache()



torch.Size([400, 3, 32, 32])


In [8]:
def accuracy(outputs, oznake):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == oznake).item() / len(preds))

class ImageClassificationBase(nn.Module):
    def training_step(self, batch):
        slike, oznake = batch 
        izlaz = self(slike)
        gubitak = func.cross_entropy(izlaz, oznake)
        return gubitak
    
    def validation_step(self, batch):
        slike, oznake = batch 
        izlaz = self(slike)
        gubitak = func.cross_entropy(izlaz, oznake)
        acc = accuracy(izlaz, oznake) 
        return {'val_loss': gubitak.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], last_lr: {:.5f}, train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['lrs'][-1], result['train_loss'], result['val_loss'], result['val_acc']))
     
    
    
def convBlock(ChannelsIn, channelsOut, pool=False):
    layers = [nn.Conv2d(ChannelsIn, channelsOut, kernel_size=3, padding=1), 
            nn.BatchNorm2d(channelsOut), 
            nn.ReLU(inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

class ResNet9(ImageClassificationBase):
    def __init__(self, channelsIn, numClasses, channelsOut):
        super().__init__()
        
        self.conv1 = convBlock(channelsIn, channelsOut)
        channelsIn = channelsOut
        self.conv2 = convBlock(channelsIn, channelsOut*2, pool=True)
        self.res1 = nn.Sequential(convBlock(channelsIn*2, channelsOut*2), convBlock(channelsIn*2, channelsOut*2))
        
        self.conv3 = convBlock(channelsIn*2, channelsOut*4, pool=True)
        self.conv4 = convBlock(channelsIn*4, channelsOut*8, pool=True)
        self.res2 = nn.Sequential(convBlock(channelsIn*8, channelsOut*8), convBlock(channelsIn*8, channelsOut*8))
        
        self.classifier = nn.Sequential(nn.MaxPool2d(4), 
                                        nn.Flatten(), 
                                        nn.Linear(512, numClasses))
        
    def forward(self, i):
        izlaz = self.conv2(self.conv1(i))
        izlaz = self.res1(izlaz) + izlaz
        izlaz = self.conv4(self.conv3(izlaz))
        izlaz = self.res2(izlaz) + izlaz
        return self.classifier(izlaz)

In [9]:
model = na_uredaj(ResNet9(3, 10, 64), uredaj)
model

ResNet9(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (res1): Sequential(
    (0): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=Tr

In [10]:
@torch.no_grad()
def procijeni(model, validDL):
    model.eval()
    outputs = [model.validation_step(batch) for batch in validDL]
    return model.validation_epoch_end(outputs)

def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def customOptimizer(model, max_lr, weightDecay, opt_func):
    return opt_func(model.parameters(), max_lr, weight_decay=weightDecay)

def jedan_ciklus(train_loader, val_loader, epohe, maximalna_stopa_ucenja, model, 
                  raspadanje=0, grad_clip=None, optimalna_funkcija=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []
    
    optimizator = optimalna_funkcija(model.parameters(), maximalna_stopa_ucenja, weight_decay=raspadanje)
    raspored = torch.optim.lr_scheduler.OneCycleLR(optimizator, maximalna_stopa_ucenja, epochs=epohe, 
                                                steps_per_epoch=len(train_loader))
    
    for epoha in range(epohe):
        model.train()
        train_losses = []
        lrs = []
        for batch in train_loader:
            gubitak = model.training_step(batch)
            train_losses.append(gubitak)
            gubitak.backward()
            
            if grad_clip: 
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
            
            optimizator.step()
            optimizator.zero_grad()
            
            lrs.append(get_lr(optimizator))
            raspored.step()
        
        result = procijeni(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoha+1, result)
        history.append(result)
    return history


In [11]:
import gc
import os
os.environ['CUDA_VISIBLE_DEVICES']='2, 3'

gc.collect()

torch.cuda.empty_cache()
history = [procijeni(model, validDL)]
history

[{'val_loss': 2.3011832237243652, 'val_acc': 0.10028846561908722}]

In [12]:
%%time
history += jedan_ciklus(trainDL, validDL, 25, 0.009, model,
                             grad_clip=0.1, 
                             raspadanje=1e-4, 
                             optimalna_funkcija=torch.optim.Adam)

Epoch [1], last_lr: 0.00073, train_loss: 1.6550, val_loss: 1.1396, val_acc: 0.5904
Epoch [2], last_lr: 0.00178, train_loss: 1.1856, val_loss: 0.8495, val_acc: 0.7024
Epoch [3], last_lr: 0.00334, train_loss: 1.0208, val_loss: 0.7900, val_acc: 0.7188
Epoch [4], last_lr: 0.00512, train_loss: 0.9409, val_loss: 0.7222, val_acc: 0.7517
Epoch [5], last_lr: 0.00684, train_loss: 0.8950, val_loss: 0.7164, val_acc: 0.7635
Epoch [6], last_lr: 0.00817, train_loss: 0.8890, val_loss: 0.9361, val_acc: 0.7224
Epoch [7], last_lr: 0.00891, train_loss: 0.8246, val_loss: 0.7251, val_acc: 0.7473
Epoch [8], last_lr: 0.00898, train_loss: 0.7335, val_loss: 0.6332, val_acc: 0.7949
Epoch [9], last_lr: 0.00884, train_loss: 0.6821, val_loss: 0.6700, val_acc: 0.7814
Epoch [10], last_lr: 0.00855, train_loss: 0.6646, val_loss: 0.4921, val_acc: 0.8338
Epoch [11], last_lr: 0.00814, train_loss: 0.6173, val_loss: 0.4278, val_acc: 0.8573
Epoch [12], last_lr: 0.00761, train_loss: 0.6185, val_loss: 0.4836, val_acc: 0.8336
E

In [13]:
torch.save(model.state_dict(), 'zavrsni.pth')

In [ ]:
model = torch.load('zavrsni.pth')
model.eval()